# IMPORT ACCESS DB TABLE TO ORACLE TABLE


CONNECTION TO ORACLE AND IMPORT REQUIRE LIBRARIES

In [ ]:
import pypyodbc
import pandas as pd
import cx_Oracle
from cx_Oracle import DatabaseError
dsn_tns = cx_Oracle.makedsn(' Hostname ', ' Port ', service_name=' Service Name ')
con = cx_Oracle.connect(user='Username', password='Password', dsn=dsn_tns)

# MS ACCESS DB CONNECTION
pypyodbc.lowercase = False
conn = pypyodbc.connect(
r"Driver={Microsoft Access Driver (*.mdb, *.accdb)};" +
r"Dbq=W:\Planning\MI\despatch\DB05-DESPATCH-ARCHIVE-039.accdb;")



print("connected to oracle db!!!")


cursor = con.cursor()

cursor.execute("""ALTER SESSION SET NLS_DATE_FORMAT = 'DD-MM-YYYY HH24:MI:SS'
                                    NLS_LANGUAGE = AMERICAN""")



IMPORT ACCESS TABLE INTO DATAFRAME

In [ ]:
# OPEN CURSOR AND EXECUTE SQL and store data into pandas dataframe
cur = conn.cursor()
SQLCommand="SELECT * FROM stock_despatch_not_avl_picking_progress"

cur.execute(SQLCommand)
df = cur.fetchall()
df1= pd.DataFrame(df)
df_p = df1.iloc[:,[0,1,2,3,4,5,6,7]]
#df_p[9]= pd.to_datetime(df_p[9]).dt.strftime('%d-%m-%Y %H:%M:%S')
df_p = df_p.astype({6:int})
#print(df_p)
#df_p.dtypes

#q='CREATE TABLE XTEMP_CANCELLED_LINES_ (HEADER_ID VARCHAR2(100),ORDER_NO VARCHAR2(100),LINE_NO VARCHAR2(100), USER_CODE VARCHAR2(100), SKU VARCHAR2(100), UPC VARCHAR2(100), TDEPT VARCHAR2(100), ORDER_QTY int, CANCEL_QTY int, CANCELED_AT date, CANCEL_TYPE VARCHAR2(100))'
#cursor.execute(q)
#con.commit()

print("Pandas dataframe created!!!")

INSERT DATAFRAME INTO ORACLE DB TABLE

In [ ]:
print('Inserting data into table....')

data=[]
batch_size= 31516.75 

for i,line in df_p.iterrows():
    sql="insert into xtemp_stock_despatch (ID,ITEM_ID,SKU,ORDER_NO,PHYSICAL_STOCK_ID,ROUTING_BARCODE,QTY,SHIPMENT_ID) values (:0,:1,:2,:3,:4,:5,:6,:7)"
    data.append((line[0],line[1],line[2],line[3],line[4],line[5],line[6],line[7]))
    if len(data) % batch_size == 0:
        cursor.executemany(sql, data)
        data = []        
    
        # the connection is not autocommitted by default, so we must commit to save our changes
        
#     if data:
#         cursor.executemany(sql, data)
        con.commit()
        conn.commit()

print('data loaded into the table successfully!!!') 